In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import jieba  # 中文分词库，百度员工开发
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
print(tf.__version__)

2.2.0


In [4]:
print('初始化一个常数张量,此处的Z表示分别三个神经元输出的最终结果，纵向排列分别为4，-4，3')
z = tf.constant([4,-4,3], dtype = tf.float32)
tf.print(z)



print('对z做softmax转换')
y_hat = tf.nn.softmax(z)
tf.print(y_hat)


print('定义类别的one_hot转换')
y = tf.one_hot(2,depth=3)
tf.print(y)


print('计算交叉熵损失')

CE = tf.keras.losses.categorical_crossentropy(y, z,  from_logits= True)
tf.print('交叉熵损失:',CE)

初始化一个常数张量,此处的Z表示分别三个神经元输出的最终结果，纵向排列分别为4，-4，3
[4 -4 3]
对z做softmax转换
[0.730879366 0.000245182717 0.26887551]
定义类别的one_hot转换
[0 0 1]
计算交叉熵损失
交叉熵损失: 1.31350684


In [5]:
print(np.log(0.26887551) * -1)

1.3135067946038521


#### 计算梯度

In [6]:
with tf.GradientTape(persistent= True) as tape:
    tape.watch([z])
    # 定义前向传播
    y = tf.one_hot(2,depth=3)
    CE = tf.keras.losses.categorical_crossentropy(y, z, from_logits=True)
    CE = tf.reduce_mean(CE)

# 求导
dCE_dz = tape.gradient(CE, [z])[0]
tf.print('求导结果：', dCE_dz)

求导结果： [0.730879366 0.000245182717 -0.73112452]


In [7]:
p = 1

In [10]:
iter_z1 = z - p * dCE_dz
tf.print(iter_z1)

[3.26912069 -4.00024509 3.7311244]


In [13]:
iter_y1 = tf.nn.softmax(iter_z1)
tf.print(iter_y1)

[0.386406571 0.000269152893 0.613324285]


In [16]:
print('计算交叉熵损失')

iter_CE1 = tf.keras.losses.categorical_crossentropy(y, iter_z1,  from_logits= True)
tf.print('迭代以后的第二轮交叉熵损失:',iter_CE1)

计算交叉熵损失
迭代以后的第二轮交叉熵损失: 0.488861471
